# find

In [1]:
import cv2
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from pyzbar.pyzbar import decode
import zxing
import math
from datetime import timedelta
from time import time, strftime, localtime
import atexit

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.style.use('grayscale')

step = 2
color_threshold = 200

def time_past(start):
    end = time()
    elapsed = end-start
    return elapsed

# open image, send it to zbar and calculate
# perspective transformations
# return arrays: located qr-codes, may be incorretly rotated;
# perspective_transforms - matrix for cv2.warpPerspective;
# rects - rectangles in whitch qr-codes licated
def locate_qr(file_name):
    codes = decode(Image.open(file_name))
    #print(codes)
    located, perspective_transforms, rects = [], [], []
    for code in codes:
        if code.type == 'QRCODE':
            img = cv2.imread(file_name, 0)
            img = img[code.rect.top:code.rect.top+code.rect.height,
                      code.rect.left:code.rect.left+code.rect.width]

            pts1 = np.float32([[code.polygon[0][0]-code.rect.left, code.polygon[0][1]-code.rect.top],
                               [code.polygon[1][0]-code.rect.left,
                                   code.polygon[1][1]-code.rect.top],
                               [code.polygon[2][0]-code.rect.left,
                                   code.polygon[2][1]-code.rect.top],
                               [code.polygon[3][0]-code.rect.left, code.polygon[3][1]-code.rect.top]])
            pts2 = np.float32([[0, 0],
                               [code.rect.height, 0],
                               [code.rect.height, code.rect.height],
                               [0, code.rect.height]])
            perspective_transform = cv2.getPerspectiveTransform(pts1, pts2)

            located.append(cv2.warpPerspective(
                img, perspective_transform, (code.rect.height, code.rect.height)))
            perspective_transforms.append(perspective_transform)
            rects.append(code.rect)
    return located, perspective_transforms, rects


# return point which can be treated as corner without pattern and estimated block size;
# given point is outer corner of pattern
def find_patternless_corner(img, pattern_y, pattern_x, dy, dx):
    pattern_size = [1, 1, 3, 1, 1]
    pattern_color = [0, 255, 0, 255, 0]
    half_height = int(img.shape[1] / 3)
    size = [0, 0, 0, 0, 0]

    y, x = pattern_y, pattern_x
    for i in range(0, len(pattern_size)):
        if abs(img[y, x] - pattern_color[i]) > color_threshold:
            return [y, x]

        prev_y, prev_x = y, x
        while abs(img[prev_y, prev_x]-int(img[y, x])) < color_threshold:
            y += dy
            x += dx
            if abs(y - pattern_y) > half_height or abs(x - pattern_x) > half_height:
                return [y, x]

        size[i] += abs(prev_y - y) / pattern_size[i]

    average_size = sum(size) / len(size)
    if max(size) - min(size) > average_size:
        return [pattern_y, pattern_x]
    else:
        return average_size


# return correctly rotated QR-code and estimated block size
def rotate_qr(img):
    size = 0

    rotation_angle = None
    for y in range(step, img.shape[0], img.shape[0]-step*2):
        for x in range(step, img.shape[1], img.shape[1]-step*2):
            rows, cols = img.shape
            dy = int(np.sign(cols / 2 - y) * step)
            dx = int(np.sign(rows / 2 - x) * step)

            tmp = find_patternless_corner(img, y, x, dy, dx)

            if type(tmp) is list:
                if tmp[0] > cols / 2 and tmp[1] < rows / 2:
                    rotation_angle = 90
                elif tmp[0] < cols / 2 and tmp[1] < rows / 2:
                    rotation_angle = 180
                elif tmp[0] < cols / 2 and tmp[1] > rows / 2:
                    rotation_angle = 270
                else:
                    rotation_angle = 0
            else:
                size += tmp

    if 0 == size:
        raise RuntimeError(
            'Block size not found, mean all corners have not patterns')
    if rotation_angle is None:
        raise RuntimeError('Paternless corner not found')

    if rotation_angle != 0:
        rotation_matrix = cv2.getRotationMatrix2D(
            (cols/2, rows/2), rotation_angle, 1)
        img = cv2.warpAffine(img, rotation_matrix, (cols, rows))
    else:
        rotation_matrix = None

    size /= 3
    return size, img, rotation_matrix


# return width and height of block
# img is correctly rotated code
# est_size is estimated block size
def find_block_size(img, est_size):
    pattern_corner = [int(est_size * 7), int(est_size * 7)]

    color = int(img[pattern_corner[0], pattern_corner[1]])
    cnt = 0
    for y in range(pattern_corner[0], img.shape[0] - pattern_corner[0], step):
        if abs(img[y, pattern_corner[1]] - color) > color_threshold:
            cnt += 1
            color = int(img[y, pattern_corner[1]])
    size = [img.shape[0] / (cnt + 13), 0]

    color = int(img[pattern_corner[0], pattern_corner[1]])
    cnt = 0
    for x in range(pattern_corner[1], img.shape[1] - pattern_corner[1], step):
        if abs(img[pattern_corner[0], x] - color) > color_threshold:
            cnt += 1
            color = int(img[pattern_corner[0], x])

    size[1] = img.shape[1] / (cnt + 13)

    if img.shape[0] == img.shape[1] and size[0] != size[1]:
        size[0] = size[1] = (size[0] + size[1]) / 2

    return size


# uses kmeans to classify colors
# return lables - lables for each block of qr-code
#        lables_to_bits - array, indexes - lables, values - 3bit arrays,
#                         each can be treated as flag for having
#                         corresponding R, G or B color on this block
def cluster_colors(color_img, size, n_clusters):
    X = []
    x = size[1] / 2
    while(x < color_img.shape[1]):
        y = size[0] / 2
        while(y < color_img.shape[0]):
            X.append(color_img[int(y), int(x)])
            y += size[0]
        x += size[1]

    X = np.array(X)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    lables = kmeans.predict(X)

    thresholds = [0, 0, 0]
    for color in range(0, len(kmeans.cluster_centers_[0])):
        for cluster_center in kmeans.cluster_centers_:
            thresholds[color] += cluster_center[color]
        thresholds[color] /= len(kmeans.cluster_centers_)

    lables_to_bits = []

    for cluster_center in kmeans.cluster_centers_:
        tmp = []
        for color in range(0, len(cluster_center)):
            if cluster_center[color] > thresholds[color]:
                tmp.append(1)
            else:
                tmp.append(0)
        lables_to_bits.append(tmp)

    return lables, lables_to_bits


# make image of qr-code for each dimension of lables_to_bit[0]
# and send it to zbar
# return data from this codes
def recover_data_from_lables(lables, lables_to_bits, qr_shape, qr_margin):
    qr = Image.new('1', qr_shape, 1)
    pix = qr.load()
    reader = zxing.BarCodeReader()
    data = []

    for color in range(0, len(lables_to_bits[0])):
        i = 0
        for x in range(qr_margin[0], qr_shape[0] - qr_margin[0]):
            for y in range(qr_margin[1], qr_shape[1] - qr_margin[1]):
                pix[y, x] = (lables_to_bits[lables[i]][color])
                i += 1

        #plt.subplot(122), plt.imshow(qr), plt.title('qr')
        zbar_code = decode(qr)

        if not zbar_code:
            qr.save('tmp.jpeg', 'jpeg')
            zxing_code = reader.decode("tmp.jpeg", True)

            if not zxing_code:
                continue
            else:
                #print(zxing_code)
                data.append(zxing_code.raw)
        else:
            #print(zbar_code)
            data.append(zbar_code[0].data)
    return data


def decode_color_qr(file_name, n_colors=8, block_size_divider=3):
    start = time()

    codes, perspective_transform, rect = locate_qr(file_name)
    data = []
    i = 0
    for code in codes:
        size, img, rotation_matrix = rotate_qr(code)

        size_true = find_block_size(img, size)
        size_true[0], size_true[1] = round(
            size_true[0] / block_size_divider), round(size_true[1] / block_size_divider)

        #print('img shape = ', img.shape, 'true size = ',
        #      img.shape[0] / (21 + 7 + 7))
        #print('block size est = ', size)
        #print('block size = ', size_true)

        color_img = cv2.imread(file_name)
        color_img = color_img[rect[i].top:rect[i].top+rect[i].height,
                              rect[i].left:rect[i].left+rect[i].width]
        color_img = cv2.warpPerspective(
            color_img, perspective_transform[i], (rect[i].height, rect[i].height))
        if rotation_matrix is not None:
            color_img = cv2.warpAffine(
                color_img, rotation_matrix, (color_img.shape[0], color_img.shape[1]))

        lables, lables_to_bits = cluster_colors(
            color_img, size_true, n_colors)

        qr_block_size = (1, 1)
        qr_margin = (int(qr_block_size[0] * 4),
                     int(qr_block_size[1] * 4))  # white area around qr-code
        qr_shape = (int(math.sqrt(len(lables)) + qr_margin[0] * 2),
                    int(math.sqrt(len(lables)) + qr_margin[1] * 2))

        data.append(recover_data_from_lables(
            lables, lables_to_bits, qr_shape, qr_margin))

        i += 1
    return data, time_past(start)


def hard_decode_colors(file_name, n_colors=8, block_size=(2, 2)):
    start = time()

    img = cv2.imread(file_name)
    #plt.subplot(223), plt.imshow(img, plt.title('img')

    lables, lables_to_bits = cluster_colors(img, block_size, n_colors)

    #print('shape = ', img.shape)
    #print('step = ', block_size)

    qr_block_size = (1, 1)
    qr_margin = (block_size[0] * 4, block_size[1]
                 * 4)  # white area around qr-code
    qr_shape = (int(math.sqrt(len(lables)) + qr_margin[0] * 2),
                int(math.sqrt(len(lables)) + qr_margin[1] * 2))

    data = recover_data_from_lables(
        lables, lables_to_bits, qr_shape, qr_margin)
    
    return data, time_past(start)


# hide

In [2]:
from PIL import Image
import qrcode
from qrcode import QRCode


def generate_qr(mess, color, version=None, filename=None):
    qr = qrcode.QRCode(
        version=version,
        error_correction=qrcode.constants.ERROR_CORRECT_Q,
        box_size=10,
        border=4,
    )
    qr.add_data(mess, optimize=0)
    if version is not None:
        qr.make(fit=True)

    img = qr.make_image(fill_color=color, back_color='white')
    if filename is not None:
        img.save(filename + '.png', 'png')
    return img


def two_color_hide(mess, filename=None):
    firstpart, secondpart = mess[:round(
        len(mess)/2)], mess[round(len(mess)/2):]

    qr_c = generate_qr(firstpart, 'cyan')
    qr_m = generate_qr(secondpart, 'magenta')
    summ = Image.merge('RGB', (qr_c.getchannel(
        0), qr_m.getchannel(1), qr_c.getchannel(2)))
    
    if filename is not None:
        summ.save(filename + '.png', 'png')
    return summ


def three_color_hide(mess, filename=None):
    firstpart, secondpart, thirdpart = mess[:round(len(
        mess)/3)], mess[round(len(mess)/3):round(len(mess)/3*2)], mess[round(len(mess)/3*2):]

    qr_r = generate_qr(thirdpart, 'cyan')
    qr_g = generate_qr(secondpart, 'magenta')
    qr_b = generate_qr(firstpart, 'yellow')
    summ = Image.merge('RGB', (qr_r.getchannel(0), qr_g.getchannel(1), qr_b.getchannel(2)))
    
    if filename is not None:
        summ.save(filename + '.png', 'png')
    return summ


# noise

In [3]:
import cv2
import os
import numpy as np

# Gaussian-distributed noise.
def gauss_noise(image, mean = 0, sigma = 1):
    row, col, ch = image.shape
    
    gauss = np.round(np.random.normal(mean, sigma, (row, col, ch)))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss.astype(np.int16)
    
    return np.clip(noisy, 0, 255).astype(np.uint8)


# Replaces random pixels with 0 or 1.
def s_p_noise(image, s_vs_p = 0.5, amount = 0.004):
    row, col, ch = image.shape
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    out[tuple(coords)] = 255

    # Pepper mode
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    out[tuple(coords)] = 0
    return out.astype(np.uint8)


# Poisson-distributed noise generated from the data.
def poisson_noise(image, peak=10):
    noisy = np.random.poisson(image / 255.0 * peak) / peak * 255
    return np.clip(noisy, 0, 255).astype(np.uint8)


# Multiplicative noise using out = image + n*image, where
# n is uniform noise with specified mean & sigma.
def speckle_noise(image, mean=0, sigma=0.1):
    row, col, ch = image.shape
    
    gauss = sigma * np.random.randn(row, col, ch) + mean
    gauss = gauss.reshape(row, col, ch).astype(np.uint16)
    noisy = image + image * gauss    
    return np.clip(noisy, 0, 255).astype(np.uint8)


## black and white qr-code test

In [47]:
def bw_make_test(
    n_iter=10,
    repeats=10,
    noise_types=['gauss', 'salt and pepper', 'poisson'],
    gauss_sigma_step=5,
    s_p_amount_step=0.002,
    poisson_peak_step=5,
    ALPHABET=np.array(list(string.ascii_lowercase + ' '))
):

    columns = ['time',
               'noise_type', 'noise_param',
               'decoded']
    data = [[None,
             None, None,
             None]]

    mess = ''.join(s for s in np.random.choice(ALPHABET, size=3 + 100))
    
    generate_qr(mess, 'black', filename = 'bw_test')
    reader = zxing.BarCodeReader()

    for noise_type in noise_types:
        print(' >> {0}'.format(noise_type))

        if 'gauss' == noise_type:
            mean = 0
            sigma_step = gauss_sigma_step
            print('\tmean = {0}, sigma_step = {1}'.format(
                mean, sigma_step))

        if 'salt and pepper' == noise_type:
            s_vs_p = 0.5
            amount_step = s_p_amount_step
            print('\ts_vs_p = {0}, amount_step = {1}'.format(
                s_vs_p, amount_step))

        if 'poisson' == noise_type:
            peak_max = 50
            peak_step = - poisson_peak_step
            print(
                '\tpeak_max = {0} peak_step = {1}'.format(peak_max, peak_step))

        if 'gauss' == noise_type:
            sigma = 0
        if 'salt and pepper' == noise_type:
             amount = 0
        if 'poisson' == noise_type:
            peak = peak_max

        step = 0
        while step < n_iter:
            log = '\tstep: {0}'.format(step)
            step += 1

            noise_param = 0.0
            if 'gauss' == noise_type:
                sigma += sigma_step
                log += ', sigma = {0}'.format(sigma)
                noise_param = sigma
            if 'salt and pepper' == noise_type:
                amount += amount_step
                log += ', amount = {0}'.format(amount)
                noise_param = amount
            if 'poisson' == noise_type:
                peak += peak_step
                if (0 >= peak):
                    print('!!! peak = {0} is less then 1, division by zero !'.format(peak))
                    break                
                log += '. peak = {0}'.format(peak)
                noise_param = peak
                
            print(log)
                
            for repeat in range(repeats):
                print('\r\t\trepeat: {1}'.format(step, repeat), end='')
                img = cv2.imread('bw_test.png')

                if 'gauss' == noise_type:
                    img = gauss_noise(img, mean, sigma)
                if 'salt and pepper' == noise_type:
                    img = s_p_noise(img, s_vs_p, amount)
                if 'poisson' == noise_type:
                    img = poisson_noise(img, peak)

                cv2.imwrite('bw_test_tmp.png', img)

                start = time()
                zxing_code = reader.decode('bw_test_tmp.png', True)
                end = time()
                time_past = end-start
                    # data_ret, time_past = hard_decode_colors(
                    #    'test_tmp.png', colors, block_size)

                qr_is_finded = 0
                if zxing_code is not None:
                    if mess in zxing_code.raw:
                        qr_is_finded = 1
                        
                data = np.append(data, [[time_past,
                                        noise_type, noise_param,
                                        qr_is_finded]], axis=0)
            print('')

    df = pd.DataFrame(data, columns=columns)
    df = df.dropna()
    df = df.astype({'time': float,
                    'noise_type': str, 'noise_param': float,
                    'decoded': int})
    return df


### run bw tests

In [ ]:
bw = bw_make_test(
    n_iter = 10,
    repeats = 50,
    noise_types=['gauss', 'salt and pepper','poisson'],
    gauss_sigma_step=5,
    s_p_amount_step=0.002,
    poisson_peak_step=5,
    ALPHABET=np.array(list(string.ascii_lowercase + ' '))
)

In [56]:
bw_g = bw[bw.noise_type == 'gauss'].groupby('noise_param').agg({'time' :[len, np.mean, np.std],
                                                                'decoded' : np.sum})
bw_g['decoded_part'] = bw_g.decoded['sum'] / bw_g.time['len']
bw_g.head(20)

time                     decoded decoded_part
              len      mean       std     sum             
noise_param                                               
5.0          50.0  0.368041  0.013946      50         1.00
10.0         50.0  0.370253  0.014457      50         1.00
15.0         50.0  0.378263  0.013989      50         1.00
20.0         50.0  0.372958  0.015698      50         1.00
25.0         50.0  0.385552  0.018204      50         1.00
30.0         50.0  0.373353  0.013749      50         1.00
35.0         50.0  0.377877  0.029569      49         0.98
40.0         50.0  0.371547  0.024979      49         0.98
45.0         50.0  0.369983  0.026667      49         0.98
50.0         50.0  0.404264  0.056275      44         0.88

In [57]:
bw_sp = bw[bw.noise_type == 'salt and pepper'].groupby('noise_param').agg({'time' :[len, np.mean, np.std],
                                                                'decoded' : np.sum})
bw_sp['decoded_part'] = bw_sp.decoded['sum'] / bw_sp.time['len']
bw_sp.head(20)

time                     decoded decoded_part
              len      mean       std     sum             
noise_param                                               
0.002        50.0  0.418622  0.074897      32         0.64
0.004        50.0  0.459600  0.075237      19         0.38
0.006        50.0  0.458070  0.069035      17         0.34
0.008        50.0  0.486719  0.056606       9         0.18
0.010        50.0  0.503944  0.050727       5         0.10
0.012        50.0  0.509575  0.035036       2         0.04
0.014        50.0  0.504069  0.035864       3         0.06
0.016        50.0  0.510185  0.018951       0         0.00
0.018        50.0  0.504220  0.016094       0         0.00
0.020        50.0  0.509736  0.030343       1         0.02

In [58]:
bw_p = bw[bw.noise_type == 'poisson'].groupby('noise_param').agg({'time' :[len, np.mean, np.std],
                                                                'decoded' : np.sum})
bw_p['decoded_part'] = bw_p.decoded['sum'] / bw_p.time['len']
bw_p.head(20)

time                     decoded decoded_part
              len      mean       std     sum             
noise_param                                               
5.0          50.0  0.526319  0.020854       0         0.00
10.0         50.0  0.521746  0.027562       1         0.02
15.0         50.0  0.532904  0.046222       3         0.06
20.0         50.0  0.486686  0.079325      16         0.32
25.0         50.0  0.484687  0.078446      17         0.34
30.0         50.0  0.476093  0.082048      19         0.38
35.0         50.0  0.437439  0.081641      30         0.60
40.0         50.0  0.441451  0.079713      29         0.58
45.0         50.0  0.411632  0.071716      38         0.76

# color qr tests

In [30]:
import string
import numpy as np
import pandas as pd


def generate_images(n_images, ALPHABET):
    image_names = []
    mess = []

    for i in range(n_images):
        image_names.append('test_'+str(i))
        tmp = ''.join(s for s in np.random.choice(ALPHABET, size=i * 9 + 100))
        mess.append(tmp)
        three_color_hide(mess[i], image_names[i])

    return image_names, mess


def make_test(
    n_images=5,
    repeats=10,
    n_iter = 10,
    noise_types=['gauss', 'salt and pepper', 'poisson'],
    gauss_sigma_step=5,
    s_p_amount_step=0.002,
    poisson_peak_step=5,
    block_size_min=1,
    block_size_max=5,
    colors=8,
    ALPHABET=np.array(list(string.ascii_lowercase + ' '))
):

    columns = ['img_name', 'decoder_type', 'time',
               'block_size', 'noise_type', 'noise_param',
               'yellow_qr', 'magenta_qr', 'cian_qr']
    data = [[None, None, None,
             None, None, None,
             None, None, None]]

    image_names, mess = generate_images(n_images, ALPHABET)

    for image in range(n_images):
        print(' >>> image: ', image_names[image])

        mess_len_third = len(mess[image])/3
        splitted_mess = [mess[image][:round(mess_len_third)],
                         mess[image][round(mess_len_third):round(
                             mess_len_third*2)],
                         mess[image][round(mess_len_third*2):]]

        for noise_type in noise_types:
            print(' >> {0}'.format(noise_type))

            if 'gauss' == noise_type:
                mean = 0
                sigma_step = gauss_sigma_step
                print('\tmean = {0}, sigma_step = {1}'.format(mean, sigma_step))

            if 'salt and pepper' == noise_type:
                s_vs_p = 0.5
                amount_step = s_p_amount_step
                print('\ts_vs_p = {0}, amount_step = {1}'.format(
                    s_vs_p, amount_step))

            if 'poisson' == noise_type:
                peak_max = 50
                peak_step = - poisson_peak_step
                print(
                    '\tpeak_max = {0} peak_step = {1}'.format(peak_max, peak_step))

            block_size_step = 1
            for block_size_axis in range(block_size_min, block_size_max + 1, block_size_step):
                block_size = (block_size_axis, block_size_axis)
                print(' > block_size = {0}'.format(block_size))

                if 'gauss' == noise_type:
                    sigma = 0
                if 'salt and pepper' == noise_type:
                    amount = 0
                if 'poisson' == noise_type:
                    peak = peak_max

                step = 0
                while step < n_iter:
                    log = '\tstep: {0}'.format(step)
                    step += 1

                    noise_param = 0.0
                    if 'gauss' == noise_type:
                        sigma += sigma_step
                        log += ', sigma = {0}'.format(sigma)
                        noise_param = sigma
                    if 'salt and pepper' == noise_type:
                        amount += amount_step
                        log += ', amount = {0}'.format(amount)
                        noise_param = amount
                    if 'poisson' == noise_type:
                        peak += peak_step
                        if (0 >= peak):
                            print('!!! peak = {0} is less then 1, division by zero !'.format(peak))
                            break                
                        log += '. peak = {0}'.format(peak)
                        noise_param = peak

                    print(log)

                    for repeat in range(repeats):
                        print('\r\t\trepeat: {1}'.format(step, repeat), end='')

                        img = cv2.imread(image_names[image] + '.png')

                        if 'gauss' == noise_type:
                            img = gauss_noise(img, mean, sigma)
                        if 'salt and pepper' == noise_type:
                            img = s_p_noise(img, s_vs_p, amount)
                        if 'poisson' == noise_type:
                            img = poisson_noise(img, peak)

                        cv2.imwrite('test_tmp.png', img)

                        data_ret, time_past = hard_decode_colors(
                            'test_tmp.png', colors, block_size)

                        qr_is_finded = [0, 0, 0]
                        if len(data_ret) != 0:
                            for i in range(len(splitted_mess)):
                                if splitted_mess[i] in data_ret:
                                    qr_is_finded[i] = 1

                        data = np.append(data, [[image_names[image], 'hard', time_past,
                                                 block_size_axis, noise_type, noise_param,
                                                 qr_is_finded[0], qr_is_finded[1], qr_is_finded[2]]], axis=0)
                    print('')

    df = pd.DataFrame(data, columns=columns)
    df = df.dropna()
    df = df.astype({'img_name': str, 'decoder_type': str, 'time': float,
                    'block_size': int, 'noise_type': str, 'noise_param': float,
                    'yellow_qr': int, 'magenta_qr': int, 'cian_qr': int})
    
    df['decoded'] = (df['yellow_qr'] + df['magenta_qr'] + df['cian_qr']) == 3
    
    return df


### run color tests

In [ ]:
df = make_test(
    n_images=1,
    repeats= 50,
    n_iter = 10,
    noise_types=['gauss'],
    gauss_sigma_step = 5,
    s_p_amount_step = 0.0005,
    poisson_peak_step = 5,
    block_size_min = 1,
    block_size_max = 10,
    colors=8,
    ALPHABET=np.array(list(string.ascii_lowercase + ' ')))

In [42]:
ga = df[df.noise_type == 'gauss'].groupby(['noise_param', 'block_size']).agg({'time' :[len, np.mean, np.std],
                                                                'yellow_qr' : np.sum,
                                                                'magenta_qr' : np.sum,
                                                                'cian_qr' : np.sum,
                                                                'decoded' : np.sum})
ga['decoded_part'] = ga.decoded['sum'] / ga.time['len']
ga.head(100)

time                     yellow_qr magenta_qr cian_qr  \
                         len      mean       std       sum        sum     sum   
noise_param block_size                                                          
5.0         1           50.0  2.217491  0.054733         0         50      50   
            3           50.0  1.037870  0.026607        50         50      50   
            5           50.0  0.958262  0.016612        50         50      50   
            7           50.0  0.944931  0.028782        50         50      50   
            9           50.0  1.066525  0.021663         0          0       0   
10.0        1           50.0  2.204752  0.044702         0         50      50   
            3           50.0  1.029868  0.019637        50         50      50   
            5           50.0  0.951823  0.017188        50         50      50   
            7           50.0  0.992735  0.030427        50         50      50   
            9           50.0  1.067880  0.028266         0          0       0   
15.0        1           50.0  2.209848  0.046284         0         50      50   
            3           50.0  1.032644  0.024127        50         50      50   
            5           50.0  0.950067  0.019364        50         50      50   
            7           50.0  0.972569  0.025261        50         50      50   
            9           50.0  1.060977  0.020403         0          0       0   
20.0        1           50.0  2.206293  0.042248         0         50      50   
            3           50.0  1.035109  0.022580        50         50      50   
            5           50.0  0.954040  0.024539        50         50      50   
            7           50.0  0.976612  0.023522        50         50      50   
            9           50.0  1.064317  0.026644         0          0       0   
25.0        1           50.0  2.249979  0.065458         0         50      50   
            3           50.0  1.032282  0.019422        50         50      50   
            5           50.0  0.957173  0.020105        50         50      50   
            7           50.0  0.981003  0.025322        50         50      50   
            9           50.0  1.064660  0.026344         0          0       0   
30.0        1           50.0  2.314652  0.070653         0         50      50   
            3           50.0  1.042319  0.026594        50         50      50   
            5           50.0  0.961224  0.023173        50         50      50   
            7           50.0  0.955598  0.021717        50         50      50   
            9           50.0  1.077566  0.027706         0          0       0   
35.0        1           50.0  2.393999  0.092638         1         45      47   
            3           50.0  1.043820  0.022344        49         50      50   
            5           50.0  0.963541  0.018790        48         50      50   
            7           50.0  0.953251  0.025198        49         50      50   
            9           50.0  1.083642  0.030175         0          0       0   
40.0        1           50.0  2.549959  0.127820         3         44      42   
            3           50.0  1.058586  0.028354        48         46      49   
            5           50.0  0.976124  0.022409        48         49      49   
            7           50.0  0.960948  0.033570        47         48      48   
            9           50.0  1.083435  0.037392         0          0       0   
45.0        1           50.0  2.910572  0.177581         1         23      23   
            3           50.0  1.108726  0.047384        42         38      44   
            5           50.0  0.990196  0.033723        45         47      41   
            7           50.0  0.989814  0.041958        36         39      43   
            9           50.0  1.112670  0.030152         0          0       0   
50.0        1           50.0  3.158070  0.227143         7          9       9   
            3           50.0  1.189791  0.057687       

In [40]:
sp = df[df.noise_type == 'salt and pepper'].groupby(['noise_param', 'block_size']).agg({'time' :[len, np.mean, np.std],
                                                                'yellow_qr' : np.sum,
                                                                'magenta_qr' : np.sum,
                                                                'cian_qr' : np.sum,
                                                                'decoded' : np.sum})
sp['decoded_part'] = sp.decoded['sum'] / sp.time['len']
sp.head(100)

time                     yellow_qr magenta_qr cian_qr  \
                         len      mean       std       sum        sum     sum   
noise_param block_size                                                          
0.0005      1           50.0  2.062844  0.071484        47         46      50   
            3           50.0  1.085772  0.020254        50         50      50   
            5           50.0  0.978129  0.032552        49         48      50   
            7           50.0  0.965938  0.033074        49         48      50   
            9           50.0  1.060582  0.020186         0          0       0   
0.0010      1           50.0  2.102259  0.093018        38         41      50   
            3           50.0  1.089053  0.033284        47         46      50   
            5           50.0  0.970554  0.030463        45         47      50   
            7           50.0  0.962911  0.024902        50         49      50   
            9           50.0  1.067738  0.024119         0          0       0   
0.0015      1           50.0  2.127143  0.117850        40         31      50   
            3           50.0  1.085779  0.033731        45         47      50   
            5           50.0  0.975473  0.035202        47         48      50   
            7           50.0  0.986092  0.033483        46         46      50   
            9           50.0  1.059829  0.019989         0          0       0   
0.0020      1           50.0  2.182603  0.117600        26         24      50   
            3           50.0  1.098699  0.037156        40         44      50   
            5           50.0  0.993685  0.033047        46         46      50   
            7           50.0  0.972767  0.029656        48         47      50   
            9           50.0  1.073651  0.026177         0          0       0   
0.0025      1           50.0  2.160133  0.111256        29         31      50   
            3           50.0  1.094962  0.040299        43         45      50   
            5           50.0  1.013844  0.037595        46         44      50   
            7           50.0  0.983502  0.036925        45         44      50   
            9           50.0  1.067176  0.024135         0          0       0   
0.0030      1           50.0  2.213832  0.105590        18         24      50   
            3           50.0  1.111733  0.044672        43         35      50   
            5           50.0  1.001353  0.050893        39         41      50   
            7           50.0  0.981291  0.032023        44         39      50   
            9           50.0  1.074660  0.023773         0          0       0   
0.0035      1           50.0  2.207042  0.101132        22         23      50   
            3           50.0  1.101748  0.043028        38         44      50   
            5           50.0  0.992235  0.031343        41         42      50   
            7           50.0  0.963878  0.037096        43         45      50   
            9           50.0  1.069066  0.020576         0          0       0   
0.0040      1           50.0  2.209771  0.126924        19         21      50   
            3           50.0  1.088446  0.050367        39         39      50   
            5           50.0  0.999769  0.039016        44         40      50   
            7           50.0  0.951711  0.031058        42         42      50   
            9           50.0  1.070506  0.024684         0          0       0   
0.0045      1           50.0  2.241010  0.104564        13         18      50   
            3           50.0  1.083277  0.045462        40         35      50   
            5           50.0  1.018180  0.043587        36         39      50   
            7           50.0  0.955766  0.036992        42         44      50   
            9           50.0  1.063587  0.019235         0          0       0   
0.0050      1           50.0  2.249203  0.087212        15         12      50   
            3           50.0  1.093624  0.051870       

In [39]:
po = df[df.noise_type == 'poisson'].groupby(['noise_param', 'block_size']).agg({'time' :[len, np.mean, np.std],
                                                                'yellow_qr' : np.sum,
                                                                'magenta_qr' : np.sum,
                                                                'cian_qr' : np.sum,
                                                                'decoded' : np.sum})
po['decoded_part'] = po.decoded['sum'] / po.time['len']
po.head(100)

time                     yellow_qr magenta_qr cian_qr  \
                         len      mean       std       sum        sum     sum   
noise_param block_size                                                          
5.0         1           50.0  3.173802  0.108818         0          0       0   
            3           50.0  1.442086  0.028872         0          0       0   
            5           50.0  1.399373  0.047415         0          0       0   
            7           50.0  1.102039  0.022410         0          0       0   
            9           50.0  1.072908  0.021565         0          0       0   
10.0        1           50.0  3.135506  0.119775         1          1       0   
            3           50.0  1.329586  0.073907        23         24      12   
            5           50.0  1.326645  0.060392        12         11       7   
            7           50.0  1.086101  0.034667         5          9       2   
            9           50.0  1.075941  0.018115         0          0       0   
15.0        1           50.0  2.908445  0.128408         9          5       7   
            3           50.0  1.213775  0.092080        37         36      36   
            5           50.0  1.166749  0.089880        33         29      31   
            7           50.0  1.062528  0.095869        42         32      33   
            9           50.0  1.077084  0.028043         0          0       0   
20.0        1           50.0  2.580562  0.166153        29         31      26   
            3           50.0  1.140934  0.053534        47         46      46   
            5           50.0  1.053398  0.074186        47         44      45   
            7           50.0  1.015107  0.069341        47         47      47   
            9           50.0  1.062798  0.020586         0          0       0   
25.0        1           50.0  2.383207  0.112429        41         43      42   
            3           50.0  1.100479  0.037965        49         48      49   
            5           50.0  1.011945  0.048149        49         50      48   
            7           50.0  0.994337  0.045828        48         50      49   
            9           50.0  1.069280  0.025616         0          0       0   
30.0        1           50.0  2.305317  0.070790        48         49      49   
            3           50.0  1.098795  0.034855        50         50      48   
            5           50.0  1.005282  0.035392        49         50      50   
            7           50.0  0.991951  0.039076        48         49      50   
            9           50.0  1.069244  0.019050         0          0       0   
35.0        1           50.0  2.249748  0.085274        49         49      48   
            3           50.0  1.083747  0.029269        50         50      50   
            5           50.0  0.995163  0.027925        50         50      50   
            7           50.0  0.989455  0.030506        50         50      50   
            9           50.0  1.066815  0.022371         0          0       0   
40.0        1           50.0  2.197721  0.066508        49         49      50   
            3           50.0  1.087827  0.026215        50         50      50   
            5           50.0  1.007590  0.037709        49         50      50   
            7           50.0  0.987915  0.028762        50         50      50   
            9           50.0  1.062182  0.018137         0          0       0   
45.0        1           50.0  2.232567  0.071668        50         50      50   
            3           50.0  1.089916  0.028050        50         50      50   
            5           50.0  0.992822  0.023160        50         50      50   
            7           50.0  0.990736  0.027211        50         50      50   
            9           50.0  1.066532  0.023093         0          0       0   

                       decoded decoded_part  
                           sum               
noise_param block_size                     

## read/write functions

In [44]:
with pd.ExcelWriter('color_qr_tests.xlsx') as writer:
    ga.to_excel(writer, sheet_name='gauss')
    po.to_excel(writer, sheet_name='poisson')
    sp.to_excel(writer, sheet_name='salt_and_pepper')

In [60]:
with pd.ExcelWriter('bw_qr_tests.xlsx') as writer:
    bw_g.to_excel(writer, sheet_name='gauss')
    bw_p.to_excel(writer, sheet_name='poisson')
    bw_sp.to_excel(writer, sheet_name='salt_and_pepper')

In [64]:
#df.to_pickle('./qr_color_test.pkl')
bw.to_pickle('./qr_bw_test.pkl')

In [ ]:
#loaded = pd.read_pickle('./qr_color_test.pkl')
loaded = pd.read_pickle('./qr_bw_test.pkl')
loaded.describe()

# etc

In [ ]:
img = cv2.imread('bw_test.png')

plt.subplot(121), plt.imshow(img), plt.title('bef')
img = gauss_noise(img, mean=0, sigma=45)
#img = s_p_noise(img, amount = 0.002)
#img = poisson_noise(img, peak=45)
#img = speckle_noise(img, mean=0, sigma=0.5)
plt.subplot(122), plt.imshow(img), plt.title('aft')

cv2.imwrite('test.png', img)

In [ ]:
o_data_ret, o_time_past = decode_color_qr('test.png', 8, 1)
print('ordinary')
print('time = ', o_time_past)
print('data length = ', len(o_data_ret),'data = ', o_data_ret)

h_data_ret, h_time_past = hard_decode_colors('test.png', 8, (5,5))
print('hard')
print('time = ', h_time_past)
print('data length = ', len(h_data_ret), 'data = ', h_data_ret)